In [1]:
import requests
import json
from tqdm import tqdm

import pandas as pd

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
https://geoscience.data.qld.gov.au/api/action/package_search?q=seismic&rows=1000&start=0

In [5]:
# set the API endpoint
api = 'https://geoscience.data.qld.gov.au/api/action/'
# construct our query
query = api + 'package_search?q=seismic&rows=1000'
# make the get request and store it in the response object
response = requests.get(query)
# view the payload as JSON
json_response = response.json()

#and get a count of results we can retrieve
total_results = json_response['result']['count']

#hack without understanding/researching the api too much
getloop = int(total_results/1000) + 1

results_list = []
for i in range(getloop):
    loopq = query + "&start=" + str(i*1000)
    response = requests.get(loopq)
    #json_response = response.json()
    
    results_list.append(json_response)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [97]:
#look at a result for the structure
results_list[0]['result']['results'][887]

{'license_title': 'Creative Commons Attribution',
 'maintainer': None,
 'creator': 'geological-survey-of-queensland',
 'relationships_as_object': [],
 'private': False,
 'maintainer_email': None,
 'num_tags': 1,
 'organization': {'description': "GSQ unlocks the value of geoscience data to enable industry success.\r\n\r\nGSQ plays a key role in improving understanding of Queensland's resource potential and attracting exploration and investment in the resources sector.\r\n\r\n#Exploration attraction\r\nGSQ creates and distributes exploration and resource investment information and presents at national and international events. We also raise awareness of Queensland's prospectivity through new geoscientific information and manage industry exploration grants.\r\n\r\n#Mineral and energy geoscience\r\nGSQ collects and provides geoscience data, information and advice on Queensland’s mineral resources and resource potential through a range of [geoscience projects and initiatives](https://www.bu

In [98]:
#each one has organisation - don't really need
#id, , author, spatial, type is useful
#resources - a list of dicts, 
#eg resource:description:
#resource description dictionary
#a resource id, resource:size, resource:, download_url, name, url, resource_type etc

In [99]:
#will try and make a dataframe
#but using the batched results, getting the ids
#turning the resource dictionaries into dataframes, concatenating
#then concatenating back at the end

In [100]:
result_ids = []
for batch in tqdm(results_list, total=len(results_list) ):
    results = batch['result']['results']
    for r in results:
        rid = r['id']
        rauthor = r['author']
        rspatial = r['spatial']
        rtype = r['type']
        
        if len(r['resources']) > 0:
            part_list = []
            for part in r['resources']:
                #modify the columns here to include what you want
                df = pd.DataFrame([part])
                df['rid'] = rid
                df['rauthor'] = rauthor
                df['rspatial'] = rspatial
                df['type'] = rtype
                part_list.append(df)
                allparts = pd.concat(part_list)
                
            result_ids.append(allparts)

dfDatasets = pd.concat(result_ids)            

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [23:27<00:00, 281.52s/it]


In [101]:
dfDatasets.to_csv(r'H:\GSQ_Seimisc_Datasets.csv', index=False)

In [63]:
#this is what a default query would look like
query_default = api + 'package_search?q=seismic

response = requests.get(query_default)
json_response = response.json()

print(json_response['result']['count'])
print(len(json_response['result']['results']) )
for x in json_response['result']['results']:
    print(x["id"])

10
4267
aaa7c8c3-d69b-4ae6-bc4b-2542301b7e6e
39bc2c60-d727-4383-b82d-6af7fdf8cf7b
2f3d761c-6506-438a-934a-87e48cc8d25e
8a1f58c4-2e61-44a5-a573-794396652ea6
55a1c3a1-107f-4a65-9bec-bef9e6eb8d6b
8157408f-83af-4515-9279-f502ae16c155
37a2fb0c-49c5-444a-a7da-4947ae9dad94
3897db72-0d38-4f11-aad4-8fa8907d20bb
4d5ae331-3cfb-4042-a4b5-1955c135cace
30b6b5fc-ba25-4bfe-a76b-c35d72ba07e0


Check how many are of type seismic, not report

In [102]:
dfDatasets.loc[dfDatasets['type'] == 'seismic']['rid'].unique().size

1290

In [103]:
dfDatasets.loc[dfDatasets['type'] == 'seismic']

,resource:description,cache_last_updated,package_id,datastore_contains_all_records_of_source_file,datastore_active,id,size,resource:format,resource:size,download_url,...,rid,rauthor,rspatial,type,original_url,resource_id,set_url_type,ignore_hash,ckan_url,task_created
0,95332_GEOMETRY_SHP,None,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,NaN,NaN,geo-spa303286-ss095332,None,ZIP,,https://geoscience.data.qld.gov.au/dataset/39b...,...,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,None,"{""type"":""Polygon"",""coordinates"":[[[150.3344387...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,Data migrated from source,None,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,NaN,NaN,geo-doc530683-ss095332,None,JSON,1 B,https://geoscience.data.qld.gov.au/dataset/39b...,...,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,None,"{""type"":""Polygon"",""coordinates"":[[[150.3344387...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,KOGAN SEISMIC SURVEY 2012 Seismic Survey: 9533...,None,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,NaN,NaN,geo-doc1040204-ss095332,None,ZIP,935.93 MB,NaN,...,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,None,"{""type"":""Polygon"",""coordinates"":[[[150.3344387...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,KOGAN SEISMIC SURVEY 2012 Seismic Survey: 9533...,None,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,NaN,NaN,geo-doc1040205-ss095332,None,ZIP,184.41 MB,NaN,...,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,None,"{""type"":""Polygon"",""coordinates"":[[[150.3344387...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,KOGAN SEISMIC SURVEY 2012 Seismic Survey: 9533...,None,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,NaN,NaN,geo-doc1040206-ss095332,None,SHP,37.59 KB,NaN,...,39bc2c60-d727-4383-b82d-6af7fdf8cf7b,None,"{""type"":""Polygon"",""coordinates"":[[[150.3344387...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Line_96-GYR_SEGD,None,046e9982-89ff-4297-ba5b-baca0bceefaf,False,False,geo-doc1302238-ss095054,None,ZIP,89.84 MB,https://geoscience.data.qld.gov.au/dataset/046...,...,046e9982-89ff-4297-ba5b-baca0bceefaf,None,"{""type"":""Polygon"",""coordinates"":[[[140.9778903...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,Line_96-GYZ_SEGD,None,046e9982-89ff-4297-ba5b-baca0bceefaf,False,False,geo-doc1302239-ss095054,None,ZIP,499.81 MB,https://geoscience.data.qld.gov.au/dataset/046...,...,046e9982-89ff-4297-ba5b-baca0bceefaf,None,"{""type"":""Polygon"",""coordinates"":[[[140.9778903...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,Line_96-GZH_SEGD,None,046e9982-89ff-4297-ba5b-baca0bceefaf,False,False,geo-doc1302240-ss095054,None,ZIP,441.86 MB,https://geoscience.data.qld.gov.au/dataset/046...,...,046e9982-89ff-4297-ba5b-baca0bceefaf,None,"{""type"":""Polygon"",""coordinates"":[[[140.9778903...",seismic,NaN,NaN,NaN,NaN,NaN,NaN
0,Line_96-GZJ_SEGD,None,046e9982-89ff-4297-ba5b-baca0bceefaf,False,False,geo-doc1302241-ss095054,None,ZIP,443.2 MB,https://geoscience.data.qld.gov.au/dataset/046...,...,046e9982-89ff-4297-ba5b-baca0bceefaf,None,"{""type"":""Polygon"",""coordinates"":[[[140.9778903...",seismic,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
dfDatasets['url'].iloc[0]

'https://gsq-prod-ckan-horizon-public.s3.ap-southeast-2.amazonaws.com/Report%2f6342%2fDocument%2f443809%2f6342.zip'

In [108]:
dfDatasets['download_url'].loc[ (dfDatasets['type'] == 'seismic') & (dfDatasets['download_url'].str.contains('geoscience') )  ]

0    https://geoscience.data.qld.gov.au/dataset/39b...
0    https://geoscience.data.qld.gov.au/dataset/39b...
0    https://geoscience.data.qld.gov.au/dataset/8a1...
0    https://geoscience.data.qld.gov.au/dataset/8a1...
0    https://geoscience.data.qld.gov.au/dataset/13d...
                           ...                        
0    https://geoscience.data.qld.gov.au/dataset/046...
0    https://geoscience.data.qld.gov.au/dataset/046...
0    https://geoscience.data.qld.gov.au/dataset/046...
0    https://geoscience.data.qld.gov.au/dataset/046...
0    https://geoscience.data.qld.gov.au/dataset/046...
Name: download_url, Length: 2423, dtype: object

In [110]:
dfDatasets['download_url'].loc[ (dfDatasets['type'] == 'seismic') & (dfDatasets['download_url'].str.contains('geoscience') )  ].iloc[0]

'https://geoscience.data.qld.gov.au/dataset/39bc2c60-d727-4383-b82d-6af7fdf8cf7b/resource/geo-spa303286-ss095332/download/%252FSurvey%25252f95332%25252fGeometry%25252f303286%25252f95332_GEOMETRY_SHP.zip'

In [111]:
download_link = 'https://gsq-horizon.s3-ap-southeast-2.amazonaws.com/SEISMIC/2D_Surveys/2010-2019/95332_KOGAN_SEISMIC_SURVEY_2012/95332_PROCESSED.ZIP'
download_response = requests.get(download_link)



In [113]:
from zipfile import ZipFile

def extract_zip(input_zip):
    input_zip=ZipFile(input_zip)
    return {name: input_zip.read(name) for name in input_zip.namelist()}

In [130]:
len(download_response.content)

981394400

In [ ]:
import io, zipfile
download_link = 'https://gsq-horizon.s3-ap-southeast-2.amazonaws.com/SEISMIC/2D_Surveys/2010-2019/95332_KOGAN_SEISMIC_SURVEY_2012/95332_PROCESSED.ZIP'
download_response = requests.get(download_link)

zips = zipfile.ZipFile(io.BytesIO(response.content))


In [142]:
dfDatasets.shape